In [1]:
def load_input(filename):
    hotspring_lines = []

    with open(filename) as f:
        for line in f:
            row, cgods = line.strip().split(" ")
            cgods_list = cgods.split(",")
            cgods_list = [int(cgod) for cgod in cgods_list]
            hotspring_lines.append({"row": row, "cgods": cgods_list})
    return hotspring_lines


hot_springs_lines = load_input("input.txt")
hot_springs_lines[:5]

[{'row': '##???#??#?????????#?', 'cgods': [11, 6]},
 {'row': '???.#??#.???', 'cgods': [1, 1, 2, 1]},
 {'row': '#????#?.???????.', 'cgods': [3, 3, 4]},
 {'row': '.#?????####.?.#?', 'cgods': [1, 1, 5, 1]},
 {'row': '?#?????????#?#?', 'cgods': [3, 2, 1, 1, 1]}]

In [2]:
def is_valid(hotspring_line):
    # .#.#.### 1,1,3
    # print(hotspring_line)
    row, cgods = hotspring_line["row"], hotspring_line["cgods"].copy()
    in_sequence = False
    for i in range(len(row)):
        spring = row[i]
        if spring == "#":
            if len(cgods) == 0:
                return False
            in_sequence = True
            cgods[0] -= 1
            if cgods[0] < 0:
                return False
        elif spring == ".":
            if in_sequence:
                if cgods[0] == 0:
                    cgods.pop(0)
                    in_sequence = False
                    continue
                else:
                    return False

    if in_sequence:
        if cgods[0] == 0:
            cgods.pop(0)
        else:
            return False

    return not bool(cgods)


def test_is_valid():
    assert is_valid({"row": ".....", "cgods": []}) == True
    assert is_valid({"row": ".....", "cgods": [1]}) == False
    assert is_valid({"row": "....#", "cgods": [1]}) == True
    assert is_valid({"row": ".....", "cgods": [1, 1]}) == False
    assert is_valid({"row": "..#..", "cgods": [1]}) == True
    assert is_valid({"row": "..#..", "cgods": [1, 1]}) == False
    assert is_valid({"row": "..##.", "cgods": [1]}) == False
    assert is_valid({"row": "#.##.", "cgods": [1, 2]}) == True
    assert is_valid({"row": "####.", "cgods": [1, 3]}) == False
    assert is_valid({"row": "#####", "cgods": [5]}) == True


test_is_valid()

In [3]:
def possible_configurations(hotspring_line):
    row, cgods = hotspring_line["row"], hotspring_line["cgods"]

    all_possible_configurations = []
    question_marks = row.count("?")
    for i in range(2**question_marks):
        binary = bin(i)[2:].zfill(question_marks)
        configuration = row
        for j in range(question_marks):
            configuration = configuration.replace("?", binary[j], 1)
        configuration = configuration.replace("0", ".")
        configuration = configuration.replace("1", "#")
        all_possible_configurations.append(configuration)

    # print(all_possible_configurations)

    sum = 0
    for configuration in all_possible_configurations:
        if is_valid({"row": configuration, "cgods": cgods}):
            sum += 1

    return sum


def test_possible_configurations():
    test = load_input("test.txt")
    # print(test)
    assert possible_configurations(test[0]) == 1
    assert possible_configurations(test[1]) == 4
    assert possible_configurations(test[2]) == 1
    assert possible_configurations(test[3]) == 1
    assert possible_configurations(test[4]) == 4
    assert possible_configurations(test[5]) == 10


test_possible_configurations()

sum([possible_configurations(row) for row in hot_springs_lines])

6949

# Part 2

In [5]:
def load_input_part2(filename):
    hotspring_lines = []

    with open(filename) as f:
        for line in f:
            row, cgods = line.strip().split(" ")
            cgods_list = cgods.split(",")
            cgods_list = [int(cgod) for cgod in cgods_list]

            row = f"{row}?{row}?{row}?{row}?{row}"
            cgods_list = cgods_list * 5

            hotspring_lines.append({"row": row, "cgods": cgods_list})
    return hotspring_lines


hot_springs_lines_part2 = load_input_part2("input.txt")
hot_springs_lines_part2[0]

{'row': '##???#??#?????????#??##???#??#?????????#??##???#??#?????????#??##???#??#?????????#??##???#??#?????????#?',
 'cgods': [11, 6, 11, 6, 11, 6, 11, 6, 11, 6]}

In [8]:
def possible_configurations_part2(hotspring_line):
    row, cgods = hotspring_line["row"], hotspring_line["cgods"].copy()
    print(row, cgods)

    possible_configurations = row.split(".")
    possible_configurations = [
        configuration
        for configuration in possible_configurations
        if configuration != ""
    ]
    print(possible_configurations)

    for configuration in possible_configurations:
        for spring in configuration:
            if spring == "?":
                pass
            elif spring == "#":
                pass

    return 1


def test_possible_configurations_part2():
    test = load_input_part2("test.txt")
    # print(test)
    assert possible_configurations_part2(test[0]) == 1
    # assert possible_configurations_part2(test[1]) == 16384
    # assert possible_configurations_part2(test[2]) == 1
    # assert possible_configurations_part2(test[3]) == 16
    # assert possible_configurations_part2(test[4]) == 2500
    # assert possible_configurations_part2(test[5]) == 506250


test_possible_configurations_part2()

???.###????.###????.###????.###????.### [1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3]
['???', '###????', '###????', '###????', '###????', '###']


In [11]:
# Reddit solution

output = 0
file = "input.txt"
lines = [line for line in open(file).read().rstrip().split("\n")]

pos_cache = {}


def return_groups(string):
    return tuple(map(int, string.split(",")))


def possibilities(springs, groups):
    global pos_cache

    if hash(str(springs) + str(groups)) in pos_cache.keys():
        return pos_cache[hash(str(springs) + str(groups))]["counter"]

    if not groups:
        if "#" not in springs:
            return 1
        return 0

    cntr = 0
    for position in range(
        len(springs) - sum(groups[1:]) + len(groups[1:]) - groups[0] + 1
    ):
        possible = "." * position + "#" * groups[0] + "."

        for spring, possible_spring in zip(springs, possible):
            if spring != possible_spring and spring != "?":
                break
        else:
            cntr += possibilities(springs[len(possible) :], groups[1:])

    pos_cache[hash(str(springs) + str(groups))] = {"counter": cntr}
    return cntr


# part 1
for line in lines:
    springs, groups = line.split()[0], return_groups(line.split()[1])
    output += possibilities(springs, groups)

print("part 1:", output)

output = 0

# part 2
for line in lines:
    springs, groups = line.split()[0], return_groups(line.split()[1])
    output += possibilities("?".join((springs,) * 5), groups * 5)

print("part 2:", output)

part 1: 6949
part 2: 51456609952403
